# Final Project Submission
## Please fill out:

* Student name: Nastaran Nazemian
* Student pace: part time
* Scheduled project review date/time: 28/4/2020 1:30PM (PS)
* Instructor name: Eli Thomas
* Blog post URL:?????????????


## Introduction

In this project, 

In [1]:
from bs4 import BeautifulSoup
import requests 
from datetime import datetime
from datetime import date
import json
import time

In [2]:
def search_movies_page(index):
    html_page = requests.get('https://www.imdb.com/search/title/?title_type=feature&release_date=2010-01-01,2020-01-01&countries=us&start=' + str(index * 50 + 1))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    movies = soup.find_all('div', "lister-item-content")
    urls = []
    for movie in movies:
        movie_url = 'https://www.imdb.com' + movie.find('h3', 'lister-item-header').find('a')['href'] 
#         print(movie_url)
        urls.append(movie_url)
    return(urls)

# url = search_movies_page(0)
#url

In [3]:
def movie_page(url):
    movie_page = requests.get(url)    
    return BeautifulSoup(movie_page.content, 'html.parser')

# this_movie_page = movie_page("https://www.imdb.com/title/tt8367814/")
#movie_page(url)

In [4]:
#def movie_id(index,movie_index)    
#     title = {}
#     movie_title_id =search_movies_page(index)[movie_index].split('/')
#     title_id = movie_title_id[-2]
#     title[Title] = title_id 
#return(title)

In [5]:
def movie_id(url):
    title = {}
    movie_title_id = url.split('/')
    title['Title_id'] = movie_title_id[-2]
    return title
#movie_id(url)

In [6]:
def boxoffice(this_movie_page):
    box_office = ['Budget','Gross USA','Cumulative Worldwide Gross','Opening Weekend USA']
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    for x in box_office:
        try:
            finance_value = this_movie_page.find('h4', string= x + ':')
            results[x] = finance_value.parent.contents[2].replace('$',"").replace(',',"").strip()
        except:
            results[x] = None
    return results
# boxoffice(this_movie_page)

In [7]:
def movie_info(this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    try:
        results['Title'] = this_movie_page.find('div', class_="title_wrapper").find('h1', class_="").contents[0].replace('\xa0','')
    except:
        results['Title'] = None
    try:
        results['Year'] = this_movie_page.find('div', class_="title_wrapper").find('a').contents[0]  
    except:
        results['Year'] = None
    try:
        results['Popularity'] = this_movie_page.find_all('div', class_ = "titleReviewBarSubItem")[-1].find('span', "subText").contents[0].strip().split(' ')[0].strip().replace(',','')
    except:
        results['Popularity'] = None
    return results
# movie_info(this_movie_page)

In [8]:
def movie_genres(this_movie_page):
    #this_movie_page = movie_page(index, movie_index)
    genres_out = {}
    genres_list = this_movie_page.find('h4', string = 'Genres:').parent.text.replace('|','').split()
    genres_out['Genre'] = genres_list[1:]
    return genres_out
# genres_out = movie_genres(this_movie_page)
# print(genres_out['Genre'] )

In [9]:
def movie_runtime(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    runtime_out = {}
    runtime = this_movie_page.find('h4',string="Runtime:").find_next_sibling('time').text.split()
    runtime_out['Runtime_min'] = runtime[0]
    return runtime_out
#movie_runtime(url)

In [10]:
def movie_release_day(this_movie_page):
    #this_movie_page = movie_page(index,movie_index)
    release_day = {}
    release_day_list = this_movie_page.find('div', class_ = "subtext").find('a', title = "See more release dates").text.split()
    date = " ".join(release_day_list[0:3])
    release_day['Release_day'] = datetime.strptime(date, '%d %B %Y').strftime("%m/%d/%Y")
    return release_day
#movie_release_day(url)

In [11]:
def rating (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index, movie_index)
    try:
        check_rating = this_movie_page.find('div', class_="ratingValue")
        rating = check_rating.find('strong')['title'].split()
        results['IMDB_Rate'] = rating[0]
        results['Rating_count'] = rating[3].replace(',','')
    except:
        results['IMDB_Rate'] = None
        results['Rating_count'] = None
        
    try:  
        check_value = this_movie_page.find('div',"metacriticScore score_mixed titleReviewBarSubItem")
        results['Meta_Score'] = check_value.find('span').text
    except:
        results['Meta_Score'] = None
    return results
#rating(url)

In [12]:
def casts_names(this_movie_page):
    roles = [('Director',2),('Writer',2), ('Star',3)]
    results = {}

            #this_movie_page = movie_page(index,movie_index)
    for role in roles:
        cast_role = (role[0]+':')

        name_list = []
        try:
            check_role = this_movie_page.find('h4',string=role[0]+':')
            if (check_role is None):
                check_role = this_movie_page.find('h4',string=role[0]+'s:')
            names = check_role.find_next_siblings('a')
            for name in names[:role[1]]:
                name_list.append(name.text)
            results[role[0]] = name_list
        except:
            results[role[0]] = None
    return results
# casts_names(this_movie_page)

In [13]:
##Find Number_User_Review, Number_Critical_Review
def review (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', "titleReviewBarItem titleReviewbarItemBorder")
    if check_value is not None:
        review = check_value.find('span', "subText")
        user = review.find('a').text.split()
        results['User_Review_Number'] = user[0].replace(',','')
        critic = review.find('a', href = "externalreviews").text.split()
        results['Critical_Review_Number'] = critic[0]
    else:
        results['User_Review_Number'] = None
        results['Critical_Review_Number'] = None
    return(results)
#review(url)

In [14]:
##Find Meta_rating,  Genera, and Release_Day---->sub_text(Tird part)
def meta_rate (this_movie_page):
    results = {}
    #this_movie_page = movie_page(index,movie_index)
    check_value = this_movie_page.find('div', class_="title_wrapper")
    if check_value is not None:
        sub_text = check_value.find('div', class_ = "subtext").text.split('|')
        meta_rating = sub_text[0].strip()
        results['Meta_Rating'] = meta_rating
    else:
        results['Meta_Rating'] = None
    
    return(results)

#meta_rate(url)

In [15]:
# this_movie_page = movie_page(url)
def one_movie_info(url):
    this_movie_page = movie_page(url)
    movie_data = {}
#     movie_function_list = [movie_id(url), 
#                            boxoffice(this_movie_page),
#                            movie_info(this_movie_page),
#                            movie_genres(this_movie_page),
#                            movie_runtime(this_movie_page),
#                            movie_release_day(this_movie_page),
#                            rating (this_movie_page),
#                            casts_names(this_movie_page),
#                            review (this_movie_page),
#                            meta_rate (this_movie_page)]
#     for function in movie_function_list:
#         movie_data.update(function)
        
    
    try:
        movie_data.update(movie_id(url))
    except:
        print ("ERROR:")
    try:
        movie_data.update(boxoffice(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_info(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_genres(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_runtime(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(movie_release_day(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(rating (this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(casts_names(this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(review (this_movie_page))
    except:
        print ("ERROR:")
    try:
        movie_data.update(meta_rate (this_movie_page))
    except:
        print ("ERROR:")
    
    return(movie_data)
# len(one_movie_info(url))  


In [16]:
# urls = search_movies_page(0)
# #print(len(urls))
# movies_data = []
# for url in urls:
# #     this_movie_page = movie_page(url)
#     one_row = one_movie_info(url)
#     movies_data.append(one_row)
#     #print(len(movies_data))
# print(len(movies_data))

In [17]:
movies_data = []
for index in range (0, 100):
    urls = search_movies_page(index)
#     print(len(urls))
    for url in urls:
        print(url)
#         time.sleep(1)
        one_row = one_movie_info(url)
        movies_data.append(one_row)
#         print(len(movies_data))
print('end', len(movies_data))
with open('data_0423_0949PM_100pages_2010_2020.json', 'w') as outfile:
    json.dump(movies_data, outfile)

https://www.imdb.com/title/tt8367814/
https://www.imdb.com/title/tt7131622/
https://www.imdb.com/title/tt8946378/
https://www.imdb.com/title/tt8579674/
https://www.imdb.com/title/tt1598778/
https://www.imdb.com/title/tt3281548/
https://www.imdb.com/title/tt2527338/
https://www.imdb.com/title/tt6673612/
https://www.imdb.com/title/tt4154796/
https://www.imdb.com/title/tt7286456/
https://www.imdb.com/title/tt7975244/
https://www.imdb.com/title/tt6189022/
https://www.imdb.com/title/tt2584384/
https://www.imdb.com/title/tt8772262/
https://www.imdb.com/title/tt8079248/
https://www.imdb.com/title/tt5727208/
https://www.imdb.com/title/tt7349950/
https://www.imdb.com/title/tt1950186/
https://www.imdb.com/title/tt2066051/
https://www.imdb.com/title/tt1302006/
https://www.imdb.com/title/tt6320628/
https://www.imdb.com/title/tt0816692/
https://www.imdb.com/title/tt1856101/
https://www.imdb.com/title/tt6105098/
https://www.imdb.com/title/tt1375666/
https://www.imdb.com/title/tt4209788/
https://www.

https://www.imdb.com/title/tt6324278/
https://www.imdb.com/title/tt8267604/
https://www.imdb.com/title/tt0385887/
https://www.imdb.com/title/tt6472976/
https://www.imdb.com/title/tt1343092/
https://www.imdb.com/title/tt2404435/
https://www.imdb.com/title/tt8364368/
https://www.imdb.com/title/tt6398184/
https://www.imdb.com/title/tt4975722/
https://www.imdb.com/title/tt2096673/
https://www.imdb.com/title/tt0780504/
https://www.imdb.com/title/tt3104988/
https://www.imdb.com/title/tt0398286/
https://www.imdb.com/title/tt5083738/
https://www.imdb.com/title/tt1276104/
https://www.imdb.com/title/tt2381941/
https://www.imdb.com/title/tt1840309/
https://www.imdb.com/title/tt4846232/
https://www.imdb.com/title/tt5439796/
https://www.imdb.com/title/tt2239822/
https://www.imdb.com/title/tt2316204/
https://www.imdb.com/title/tt2481498/
https://www.imdb.com/title/tt1673434/
https://www.imdb.com/title/tt0369610/
https://www.imdb.com/title/tt5028340/
https://www.imdb.com/title/tt6823368/
https://www.

https://www.imdb.com/title/tt5719748/
https://www.imdb.com/title/tt8526872/
https://www.imdb.com/title/tt2245084/
https://www.imdb.com/title/tt1458175/
https://www.imdb.com/title/tt6343314/
https://www.imdb.com/title/tt2582846/
https://www.imdb.com/title/tt8385474/
https://www.imdb.com/title/tt1790864/
https://www.imdb.com/title/tt2397535/
https://www.imdb.com/title/tt1413492/
https://www.imdb.com/title/tt4034228/
https://www.imdb.com/title/tt1935859/
https://www.imdb.com/title/tt1213641/
https://www.imdb.com/title/tt7547410/
https://www.imdb.com/title/tt0435761/
https://www.imdb.com/title/tt2872732/
https://www.imdb.com/title/tt5362988/
https://www.imdb.com/title/tt1408101/
https://www.imdb.com/title/tt4160708/
https://www.imdb.com/title/tt1365519/
https://www.imdb.com/title/tt0815236/
ERROR:
https://www.imdb.com/title/tt1615160/
https://www.imdb.com/title/tt4532826/
https://www.imdb.com/title/tt1454468/
https://www.imdb.com/title/tt3332064/
https://www.imdb.com/title/tt1333125/
https

https://www.imdb.com/title/tt9134216/
https://www.imdb.com/title/tt4572514/
https://www.imdb.com/title/tt3717490/
https://www.imdb.com/title/tt1428538/
https://www.imdb.com/title/tt1133985/
https://www.imdb.com/title/tt2788710/
https://www.imdb.com/title/tt1855199/
https://www.imdb.com/title/tt2180411/
https://www.imdb.com/title/tt1411697/
https://www.imdb.com/title/tt8328716/
https://www.imdb.com/title/tt2053463/
https://www.imdb.com/title/tt1706593/
https://www.imdb.com/title/tt1499658/
https://www.imdb.com/title/tt3065204/
https://www.imdb.com/title/tt1854564/
https://www.imdb.com/title/tt6772950/
https://www.imdb.com/title/tt2377322/
https://www.imdb.com/title/tt3165612/
https://www.imdb.com/title/tt1663662/
https://www.imdb.com/title/tt1430132/
https://www.imdb.com/title/tt0964517/
https://www.imdb.com/title/tt3201640/
https://www.imdb.com/title/tt5294518/
https://www.imdb.com/title/tt4257926/
https://www.imdb.com/title/tt0983193/
https://www.imdb.com/title/tt2649554/
https://www.

https://www.imdb.com/title/tt1220634/
https://www.imdb.com/title/tt9425078/
https://www.imdb.com/title/tt2103281/
https://www.imdb.com/title/tt4786282/
https://www.imdb.com/title/tt5913798/
https://www.imdb.com/title/tt1293847/
https://www.imdb.com/title/tt1321511/
https://www.imdb.com/title/tt1620935/
https://www.imdb.com/title/tt7043012/
https://www.imdb.com/title/tt1001526/
https://www.imdb.com/title/tt3774114/
https://www.imdb.com/title/tt1583421/
https://www.imdb.com/title/tt6957966/
https://www.imdb.com/title/tt1666801/
https://www.imdb.com/title/tt1764651/
https://www.imdb.com/title/tt1155076/
https://www.imdb.com/title/tt1424381/
https://www.imdb.com/title/tt1490017/
https://www.imdb.com/title/tt2304933/
https://www.imdb.com/title/tt3774802/
https://www.imdb.com/title/tt1284575/
https://www.imdb.com/title/tt0427152/
https://www.imdb.com/title/tt2639336/
https://www.imdb.com/title/tt9000224/
https://www.imdb.com/title/tt2106651/
https://www.imdb.com/title/tt3416532/
https://www.

...

https://www.imdb.com/title/tt1663187/
ERROR:
https://www.imdb.com/title/tt3703148/
https://www.imdb.com/title/tt5839426/
https://www.imdb.com/title/tt2112277/
https://www.imdb.com/title/tt6450428/
https://www.imdb.com/title/tt2317524/
https://www.imdb.com/title/tt2308682/
ERROR:
https://www.imdb.com/title/tt2758890/
https://www.imdb.com/title/tt4286760/
ERROR:
https://www.imdb.com/title/tt5304688/
https://www.imdb.com/title/tt1707391/
https://www.imdb.com/title/tt7548732/
https://www.imdb.com/title/tt3569356/
https://www.imdb.com/title/tt3449200/
https://www.imdb.com/title/tt8748236/
https://www.imdb.com/title/tt3120314/
https://www.imdb.com/title/tt6942194/
https://www.imdb.com/title/tt1785669/
https://www.imdb.com/title/tt6598072/
https://www.imdb.com/title/tt3343136/
https://www.imdb.com/title/tt1810697/
https://www.imdb.com/title/tt2201211/
https://www.imdb.com/title/tt4510398/
https://www.imdb.com/title/tt7316576/
https://www.imdb.com/title/tt5806558/
ERROR:
https://www.imdb.com/t

ERROR:
https://www.imdb.com/title/tt1599296/
https://www.imdb.com/title/tt3503840/
https://www.imdb.com/title/tt7222228/
https://www.imdb.com/title/tt11191124/
https://www.imdb.com/title/tt1600438/
https://www.imdb.com/title/tt6469960/
https://www.imdb.com/title/tt3253232/
https://www.imdb.com/title/tt3104930/
https://www.imdb.com/title/tt4538670/
https://www.imdb.com/title/tt5030402/
https://www.imdb.com/title/tt5456798/
https://www.imdb.com/title/tt2421416/
https://www.imdb.com/title/tt4191486/
https://www.imdb.com/title/tt4577344/
https://www.imdb.com/title/tt7694738/
https://www.imdb.com/title/tt1516009/
https://www.imdb.com/title/tt2737926/
https://www.imdb.com/title/tt3564200/
https://www.imdb.com/title/tt2166616/
https://www.imdb.com/title/tt4002772/
https://www.imdb.com/title/tt6843538/
ERROR:
https://www.imdb.com/title/tt7099566/
https://www.imdb.com/title/tt2340650/
https://www.imdb.com/title/tt2306783/
ERROR:
https://www.imdb.com/title/tt2825230/
https://www.imdb.com/title/t

https://www.imdb.com/title/tt3142366/
https://www.imdb.com/title/tt1303803/
ERROR:
https://www.imdb.com/title/tt10504904/
https://www.imdb.com/title/tt2877192/
ERROR:
https://www.imdb.com/title/tt1774591/
https://www.imdb.com/title/tt4065212/
https://www.imdb.com/title/tt1850418/
https://www.imdb.com/title/tt2313780/
https://www.imdb.com/title/tt7493366/
https://www.imdb.com/title/tt2463154/
https://www.imdb.com/title/tt1628055/
https://www.imdb.com/title/tt1533058/
https://www.imdb.com/title/tt2492344/
https://www.imdb.com/title/tt1712573/
https://www.imdb.com/title/tt3125220/
https://www.imdb.com/title/tt2507238/
https://www.imdb.com/title/tt2190492/
https://www.imdb.com/title/tt4955162/
ERROR:
https://www.imdb.com/title/tt8239736/
https://www.imdb.com/title/tt4567486/
https://www.imdb.com/title/tt3367686/
https://www.imdb.com/title/tt6156656/
https://www.imdb.com/title/tt1619277/
ERROR:
https://www.imdb.com/title/tt5855164/
https://www.imdb.com/title/tt2279922/
ERROR:
https://www.im

https://www.imdb.com/title/tt2312184/
https://www.imdb.com/title/tt3717192/
ERROR:
https://www.imdb.com/title/tt4729990/
https://www.imdb.com/title/tt5450040/
https://www.imdb.com/title/tt1705125/
https://www.imdb.com/title/tt3145008/
ERROR:
ERROR:
https://www.imdb.com/title/tt2145637/
ERROR:
https://www.imdb.com/title/tt5361488/
https://www.imdb.com/title/tt5636234/
https://www.imdb.com/title/tt4375268/
https://www.imdb.com/title/tt3760966/
https://www.imdb.com/title/tt5215566/
ERROR:
https://www.imdb.com/title/tt3139538/
https://www.imdb.com/title/tt3104818/
https://www.imdb.com/title/tt3863632/
https://www.imdb.com/title/tt3724490/
https://www.imdb.com/title/tt5220252/
https://www.imdb.com/title/tt2375763/
ERROR:
https://www.imdb.com/title/tt5605076/
https://www.imdb.com/title/tt5167934/
https://www.imdb.com/title/tt7562932/
https://www.imdb.com/title/tt6057032/
https://www.imdb.com/title/tt0970926/
ERROR:
https://www.imdb.com/title/tt3962984/
https://www.imdb.com/title/tt5020744/
h

https://www.imdb.com/title/tt3339534/
https://www.imdb.com/title/tt4669408/
ERROR:
ERROR:
https://www.imdb.com/title/tt2078718/
https://www.imdb.com/title/tt2190467/
https://www.imdb.com/title/tt3339354/
https://www.imdb.com/title/tt3705822/
https://www.imdb.com/title/tt6341668/
https://www.imdb.com/title/tt6302164/
https://www.imdb.com/title/tt7919180/
https://www.imdb.com/title/tt5963314/
ERROR:
https://www.imdb.com/title/tt5612564/
https://www.imdb.com/title/tt5539284/
https://www.imdb.com/title/tt5126654/
https://www.imdb.com/title/tt1792647/
ERROR:
https://www.imdb.com/title/tt2076346/
https://www.imdb.com/title/tt2654572/
https://www.imdb.com/title/tt2724236/
https://www.imdb.com/title/tt2014346/
https://www.imdb.com/title/tt5655604/
https://www.imdb.com/title/tt1385543/
https://www.imdb.com/title/tt5813010/
https://www.imdb.com/title/tt2120025/
https://www.imdb.com/title/tt2503954/
https://www.imdb.com/title/tt4454930/
https://www.imdb.com/title/tt1298594/
ERROR:
https://www.imd

In [18]:
import re
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
df = pd.DataFrame(movies_data)
df.head()


,Title_id,Budget,Gross USA,Cumulative Worldwide Gross,Opening Weekend USA,Title,Year,Popularity,Genre,Runtime_min,Release_day,IMDB_Rate,Rating_count,Meta_Score,Director,Writer,Star,User_Review_Number,Critical_Review_Number,Meta_Rating
0,tt8367814,22000000,36296853,114996853,10651884,The Gentlemen,2019,6,"[Action, Comedy, Crime]",113,01/24/2020,8.0,111677,51,[Guy Ritchie],"[Guy Ritchie, Ivan Atkinson]","[Matthew McConaughey, Charlie Hunnam, Michelle...",869,231,R
1,tt7131622,90000000,142502728,374343626,41082018,Once Upon a Time... in Hollywood,2019,8,"[Comedy, Drama]",161,07/26/2019,7.7,445417,None,[Quentin Tarantino],[Quentin Tarantino],"[Leonardo DiCaprio, Brad Pitt, Margot Robbie]",4811,594,R
2,tt8946378,40000000,165363234,309232797,26769548,Knives Out,2019,10,"[Comedy, Crime, Drama, Mystery, Thriller]",131,11/27/2019,7.9,279465,None,[Rian Johnson],[Rian Johnson],"[Daniel Craig, Chris Evans, Ana de Armas]",1822,417,PG-13
3,tt8579674,95000000,159227644,368027644,576216,1917,2019,13,"[Drama, War]",119,01/10/2020,8.3,285988,None,[Sam Mendes],"[Sam Mendes, Krysty Wilson-Cairns]","[Dean-Charles Chapman, George MacKay, Daniel M...",2465,447,R
4,tt1598778,60000000,75658097,136515867,22403596,Contagion,2011,15,"[Action, Drama, Thriller]",106,09/09/2011,6.7,257228,None,[Steven Soderbergh],[Scott Z. Burns],"[Matt Damon, Kate Winslet, Jude Law]",684,255,PG-13


In [20]:
df.describe()

TypeError: unhashable type: 'list'

In [ ]:
df.info()
df.shape
